In [19]:
pip install pyspark


In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [21]:

# Step 1: Start Spark session
spark = SparkSession.builder.appName("CleanUncleanFinancialData").getOrCreate()

In [22]:
# Step 2: Load unclean dataset
df = spark.read.option("header", True).option("inferSchema", True).csv("/content/unclean_financial_data_extended_275k (1).csv")


In [23]:
from pyspark.sql.functions import col, sum

# Count nulls for each column
null_counts = df.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
])

# Show the result
null_counts.show()


+-------+------+---+----------+----------+---------+----+--------------+---------+---------+---------+----------+-------------+---------+----------+---------+-------------+--------------------------+---------------+-----------------+---------------------------+---------------------------+----------------------------+-------------------------------+---------------------------+----------------------------+---------------------------+-------------------------------+-----------------------+--------------------+----------------+----------+-----------+
|user_id|Income|Age|Dependents|Occupation|City_Tier|Rent|Loan_Repayment|Insurance|Groceries|Transport|Eating_Out|Entertainment|Utilities|Healthcare|Education|Miscellaneous|Desired_Savings_Percentage|Desired_Savings|Disposable_Income|Potential_Savings_Groceries|Potential_Savings_Transport|Potential_Savings_Eating_Out|Potential_Savings_Entertainment|Potential_Savings_Utilities|Potential_Savings_Healthcare|Potential_Savings_Education|Potential_Sa

In [24]:
total_rows = df.count()

null_percentages = df.select([
    (sum(col(c).isNull().cast("int")) / total_rows * 100).alias(c) for c in df.columns
])

null_percentages.show()


+-------+------+---+----------+----------+---------+----+--------------+---------+---------+---------+----------+-------------+---------+----------+---------+-------------+--------------------------+---------------+-----------------+---------------------------+---------------------------+----------------------------+-------------------------------+---------------------------+----------------------------+---------------------------+-------------------------------+-----------------------+--------------------+----------------+-----------------+-----------+
|user_id|Income|Age|Dependents|Occupation|City_Tier|Rent|Loan_Repayment|Insurance|Groceries|Transport|Eating_Out|Entertainment|Utilities|Healthcare|Education|Miscellaneous|Desired_Savings_Percentage|Desired_Savings|Disposable_Income|Potential_Savings_Groceries|Potential_Savings_Transport|Potential_Savings_Eating_Out|Potential_Savings_Entertainment|Potential_Savings_Utilities|Potential_Savings_Healthcare|Potential_Savings_Education|Poten

In [25]:
df.show()

+-------+------------------+---+----------+-------------+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------------+------------------+------------------+---------------------------+---------------------------+----------------------------+-------------------------------+---------------------------+----------------------------+---------------------------+-------------------------------+-----------------------+--------------------+----------------+------------------+-----------+
|user_id|            Income|Age|Dependents|   Occupation|City_Tier|              Rent|    Loan_Repayment|         Insurance|         Groceries|         Transport|        Eating_Out|     Entertainment|         Utilities|        Healthcare|         Education|     Miscellaneous|Desired_Savings_Percentage|   Desired_Savings| Disposab

In [27]:
# 🧹 Drop 'random_note' column if it exists
if "random_note" in df.columns:
    df = df.drop("random_note")

# 📏 Get number of rows
num_rows = df.count()

# ❌ Drop columns with more than 40% null values
null_ratio = df.select([(1 - (sum(col(c).isNotNull().cast("int")) / num_rows)).alias(c) for c in df.columns]).first().asDict()
cols_to_drop = [col_name for col_name, ratio in null_ratio.items() if ratio > 0.4]
df = df.drop(*cols_to_drop)

# 🧼 Drop remaining rows with nulls
df = df.na.drop()

# 🚮 Drop duplicate rows
df = df.dropDuplicates()

# 💾 Save the cleaned dataset
df.write.option("header", True).mode("overwrite").csv("final_cleaned_original_dataset.csv")

In [11]:
# Step 4: Drop columns with >40% nulls
total_count = df.count()
null_ratios = df.select([
    (1 - (sum(col(c).isNotNull().cast("int")) / total_count)).alias(c) for c in df.columns
]).first().asDict()
columns_to_drop = [col for col, null_pct in null_ratios.items() if null_pct > 0.4]
df = df.drop(*columns_to_drop)

In [29]:
 df = df.drop('random_note')

In [30]:
df.show()

+-------+------------------+---+----------+-------------+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------------+------------------+------------------+---------------------------+---------------------------+----------------------------+-------------------------------+---------------------------+----------------------------+---------------------------+-------------------------------+-----------------------+--------------------+----------------+
|user_id|            Income|Age|Dependents|   Occupation|City_Tier|              Rent|    Loan_Repayment|         Insurance|         Groceries|         Transport|        Eating_Out|     Entertainment|         Utilities|        Healthcare|         Education|     Miscellaneous|Desired_Savings_Percentage|   Desired_Savings| Disposable_Income|Potential_Savings_Gro

In [32]:
from pyspark.sql.functions import col, sum

# Count nulls for each column
null_counts = df.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
])

# Show the result
null_counts.show()

+-------+------+---+----------+----------+---------+----+--------------+---------+---------+---------+----------+-------------+---------+----------+---------+-------------+--------------------------+---------------+-----------------+---------------------------+---------------------------+----------------------------+-------------------------------+---------------------------+----------------------------+---------------------------+-------------------------------+-----------------------+--------------------+----------------+
|user_id|Income|Age|Dependents|Occupation|City_Tier|Rent|Loan_Repayment|Insurance|Groceries|Transport|Eating_Out|Entertainment|Utilities|Healthcare|Education|Miscellaneous|Desired_Savings_Percentage|Desired_Savings|Disposable_Income|Potential_Savings_Groceries|Potential_Savings_Transport|Potential_Savings_Eating_Out|Potential_Savings_Entertainment|Potential_Savings_Utilities|Potential_Savings_Healthcare|Potential_Savings_Education|Potential_Savings_Miscellaneous|Tot

In [17]:
df = df.na.drop()

In [34]:
# Get number of rows
num_rows = df.count()

# Get number of columns
num_columns = len(df.columns)

print(f" Number of Rows: {num_rows}")
print(f" Number of Columns: {num_columns}")

 Number of Rows: 250000
 Number of Columns: 31


In [35]:
# 💾 Save the cleaned dataset
df.write.option("header", True).mode("overwrite").csv("final_cleaned_original_dataset.csv")
